In [457]:
import logging
import collections
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open 
import pandas as pd
import numpy as np
from numpy import random
from collections import Counter
import heapq #check n largest
import json
import warnings
from sqlalchemy import create_engine 
import spacy
import nltk
import seaborn as sns

In [458]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [459]:
bath_path = r''

computer_security_and_networks = json.load(open(bath_path + r'computer-security-and-networks.json', encoding='utf8'))
algorithms = json.load(open(bath_path + r'algorithms.json', encoding='utf8'))
cloud_computing = json.load(open(bath_path + r'cloud-computing.json', encoding='utf8'))
data_management = json.load(open(bath_path + r'data-management.json', encoding='utf8'))
design_and_product = json.load(open(bath_path + r'design-and-product.json', encoding='utf8'))
electrical_engineering = json.load(open(bath_path + r'electrical-engineering.json', encoding='utf8'))
math_and_logic = json.load(open(bath_path + r'math-and-logic.json', encoding='utf8'))
mobile_and_web_development = json.load(open(bath_path + r'mobile-and-web-development.json', encoding='utf8'))
networking = json.load(open(bath_path + r'networking.json', encoding='utf8'))
probability_and_statistics = json.load(open(bath_path + r'probability-and-statistics.json', encoding='utf8'))
security = json.load(open(bath_path + r'security.json', encoding='utf8'))
machine_learning = json.load(open(bath_path + r'machine-learning.json', encoding='utf8'))
data_analysis = json.load(open(bath_path + r'data-analysis.json', encoding='utf8'))
software_development = json.load(open(bath_path + r'software-development.json', encoding='utf8'))

all_data = computer_security_and_networks + \
algorithms + \
cloud_computing + \
data_management + \
design_and_product + \
electrical_engineering + \
math_and_logic + \
mobile_and_web_development + \
networking + \
probability_and_statistics + \
security + \
machine_learning + \
data_analysis + \
software_development

In [460]:
len(all_data)

881

In [461]:
title=[]; week_data=[]; target_audience=[]; created_by=[]; teach_by=[]
week_data_title=[]; course_info=[]; department=[]; week_data_desc=[]
package_num=[];package_name=[]; level=[]; rating=[]; index=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    for i in range(len(all_data[n]['teach_by'])):
            title.append(all_data[n]['title'])
            target_audience.append(all_data[n]['target_audience'])
            created_by.append(all_data[n]['created_by'])
            course_info.append(all_data[n]['course_info'])
            package_num.append(all_data[n]['package_num'])
            package_name.append(all_data[n]['package_name'])
            level.append(all_data[n]['level'])
            rating.append(all_data[n]['rating'])
            teach_by.append(list((all_data[n]['teach_by'][i].values()))[0])  
            department.append(list((all_data[n]['teach_by'][i].values()))[1])

In [462]:
data = pd.DataFrame({'title': title,
                    'target_audience': target_audience,
                    'created_by' : created_by,  
                    'course_info': course_info,
                    'teach_by': teach_by,
                     'department': department,
                     'level': level,
                     'rating': rating,
                     'package_num': package_num,
                     'package_name': package_name          
                    },
                    columns = ['title', 'course_info', 'target_audience', 'created_by','teach_by',
                               'department','package_num','package_name', 'level','rating'])

In [463]:
data.head()

,title,course_info,target_audience,created_by,teach_by,department,package_num,package_name,level,rating
0,Bitcoin and Cryptocurrency Technologies,About this course: To really understand what i...,None,Princeton University,Arvind Narayanan,Computer Science,None,None,None,4.7
1,Cryptography I,About this course: Cryptography is an indispen...,None,Stanford University,Dan Boneh,Computer Science,None,None,None,4.8
2,Google Cloud Platform Fundamentals: Core Infra...,About this course: <<< ATTENTION >>> Learners ...,Who is this class for: This specialization is ...,Google Cloud,Google Cloud Training,None,1,Architecting with Google Cloud Platform Specia...,Intermediate,4.6
3,G Suite Administrator Fundamentals,About this course: G Suite Administrator Funda...,Who is this class for: This class is intended ...,Google Cloud,Google Cloud Training,None,None,None,Beginner,4.5
4,Usable Security,About this course: This course focuses on how ...,None,"University of Maryland, College Park",Jennifer Golbeck,Human-Computer Interaction Lab,1,Cybersecurity Specialization,None,4.5


In [464]:
data =  data.drop_duplicates()

In [465]:
for i, row in data.iterrows():
    if not isEnglish(row.title):
        data.drop(i, inplace=True)

In [466]:
len(data.title.unique())

608

In [467]:
title=[]; created_by=[]; course_info=[]
data_title=[]; data_desc=[]; data_video=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    week_data_title=[];  week_data_desc=[];week_data_video=[]
    title.append(all_data[n]['title'])
    created_by.append(all_data[n]['created_by'])
    for i in range(len(all_data[n]['week_data'])):
        week_data_desc.append(all_data[n]['week_data'][i]['description'])
        week_data_title.append(all_data[n]['week_data'][i]['title'])
        week_data_video += all_data[n]['week_data'][i]['video']
    data_title.append(week_data_title)
    data_desc.append(week_data_desc)
    data_video.append(week_data_video)

In [468]:
data_created_by = pd.DataFrame({'title': title,
                    'created_by':created_by},
                    columns = ['title', 'created_by'])

data_detail = pd.DataFrame({     
                    'week_data_desc': data_desc,
                    'week_data_title': data_title,
                    'week_data_video': data_video
                    },
                    columns = ['week_data_desc', 'week_data_title','week_data_video'])


In [469]:
data_created_by.drop_duplicates(inplace=True)

In [470]:
len(data_created_by)

637

In [471]:
for i, row in data_created_by.iterrows():
    if not isEnglish(row.title):
        data_created_by.drop(i, inplace=True)

In [472]:
data_detail = pd.merge(data_created_by, data_detail, left_index=True, right_index=True, how="left")

In [475]:
for i, row in data_detail.iterrows():
    if len(row.week_data_video) == 0:
        data_detail.drop(i, inplace=True)

In [476]:
data_detail = data_detail.reset_index(drop=True)

In [477]:
nlp = spacy.load('en')

In [478]:
week_video_list = []
for video_data in data_detail.week_data_video:
    video_str = ''
    x = ' '.join(video_data)
    doc = nlp(x.lower())
    for token in doc:
        if not token.is_stop and token.is_alpha:
            video_str += token.lemma_
            video_str += ' '
    week_video_list.append(video_str)

In [479]:
data_detail['week_data_video_str'] = pd.Series(week_video_list)
data_detail.dropna(inplace=True)
data_detail = data_detail.reset_index(drop=True)

In [481]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorize = TfidfVectorizer(max_df=150)
bows = vectorize.fit(data_detail.week_data_video_str)
document_matrix = bows.transform(data_detail.week_data_video_str).toarray()

In [482]:
document_matrix.shape

(596, 8733)

In [484]:
f = bows.get_feature_names()

In [486]:
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(document_matrix)

In [550]:
data_detail.loc[np.argsort(result[20])[::-1], :]

,title,created_by,week_data_desc,week_data_title,week_data_video,week_data_video_str,category
20,Introduction to Cybersecurity for Business,University of Colorado System,[Understand the basics of computer security fo...,"[What is Practical Computer Security, The CIA ...","[Welcome!, About the Instructor, Glossary of C...",welcome instructor glossary computer security ...,5
13,Information Security: Context and Introduction,University of London,"[, This module introduces cryptography, which ...","[Introduction to Information Security, Introdu...","[An overview, What are you hoping to achieve i...",overview hop achieve course information securi...,5
6,Cybersecurity and Its Ten Domains,University System of Georgia,[Checkout our marquee speaker. She's the Chief...,"[Welcome and Course Overview, Introduction to ...","[Welcome to ""Cybersecurity and Its Ten Domains...",welcome cybersecurity domain welcome course im...,5
408,Cyber Security in Manufacturing,"University at Buffalo, The State University of...",[The purpose of this module is to introduce yo...,[Introduction to Digital Manufacturing Securit...,"[Course Introduction, Digital Manufacturing Se...",course introduction digital manufacture securi...,6
49,Enterprise and Infrastructure Security,New York University Tandon School of Engineering,[This module includes an introduction to many ...,"[Security Awareness, Compliance, Assessments, ...",[Introduction: What You Will Learn from This C...,introduction learn course cyber security assig...,5
588,Software Requirements Prioritization: Risk Ana...,University of Colorado System,[Once requirements have been somewhat/mostly d...,"[Risk Analysis, Assessment and Prioritization,...","[Intro to Risk Analysis, Assessment, and Prior...",intro risk analysis assessment prioritization ...,1
79,The Business of Cybersecurity Capstone,University System of Georgia,[This module provides a welcome to the course ...,[Welcome to the Management of Information Secu...,"[An overview of this course, Welcome to securi...",overview course welcome security management se...,5
33,Enterprise System Management and Security,University of Colorado System,[Welcome and Introduction to Enterprise System...,[It's not that different from my home computer...,"[Welcome!, About this course, Resources for bu...",welcome course resource build test environment...,1
1,Cryptography I,Stanford University,[Week 1. This week's topic is an overview of ...,"[Course overview and stream ciphers, Block Cip...","[Lecture slides for all six weeks, Course over...",lecture slide week course overview additional ...,5
38,Cyber Threats and Attack Vectors,University of Colorado System,[Welcome! This week we'll explore users and us...,[THREATS AND ATTACK VECTORS IN COMPUTER SECURI...,"[Introduction, About this course, About the In...",introduction course instructor digital identit...,5


In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(100)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X = lsa.fit_transform(document_matrix)
inertia_list = []  
for i in range(2,15):
    kmeans = KMeans(n_clusters=i).fit(X)
    inertia_list.append(kmeans.inertia_)

import matplotlib.pyplot as plt
plt.plot(range(2,15), inertia_list, linestyle='-', marker='o')
plt.show() 

In [445]:
![image.png](attachment:image.png)

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


In [14]:
#IsDuplicated = data_to_mysql.duplicated()

In [31]:
# duplicate_course=0
# for i in range(len(data_to_mysql)):
#     if IsDuplicated[i] == True:
#         duplicate_course=duplicate_course+1
#         #print(i)
# print(duplicate_course)

167


In [30]:
#data = data_to_mysql.drop_duplicates()

In [31]:
#data_to_mysql.head()

### --------------------------------------------------------------------------------

In [458]:
### Testing

In [464]:
# v=[]
# for n in range(len(all_data)):
#     for j in range(len(all_data[n]['week_data'])):     
#         v.append(list((all_data[n]['week_data'][j]).values())[0])
#         #a.append(pd.Series(v).str.cat(sep='')) 

In [385]:
# v= pd.Series(v).str.cat(sep='')